In [1]:
from utils.dataframes import *

meter_usage

,service_point_id,meter_id,interval_end_datetime,meter_channel,kwh,account_number
0,2300822246,L108605388,10/1/2022 12:00:00 AM,10,0.594,30010320353
1,2300822246,L108605388,10/1/2022 12:15:00 AM,10,0.101,30010320353
2,2300822246,L108605388,10/1/2022 12:30:00 AM,10,0.104,30010320353
3,2300822246,L108605388,10/1/2022 12:45:00 AM,10,0.106,30010320353
4,2300822246,L108605388,10/1/2022 1:00:00 AM,10,0.099,30010320353
...,...,...,...,...,...,...
500275,2300588897,L108607371,9/30/2021 7:00:00 PM,10,1.242,35012790198
500276,2300588897,L108607371,9/30/2021 8:00:00 PM,10,1.202,35012790198
500277,2300588897,L108607371,9/30/2021 9:00:00 PM,10,1.186,35012790198
500278,2300588897,L108607371,9/30/2021 10:00:00 PM,10,1.150,35012790198


In [5]:
from utils.runtime import connect_to_db

electric_brew = connect_to_db()

electric_brew.execute("SELECT * FROM meter_usage").fetchdf()

,service_point_id,meter_id,interval_end_datetime,meter_channel,kwh,account_number
0,2300588853,L123057647,10/1/2022 12:00:00 AM,10,0.043,35012787756
1,2300588853,L123057647,10/1/2022 12:15:00 AM,10,0.040,35012787756
2,2300588853,L123057647,10/1/2022 12:30:00 AM,10,0.045,35012787756
3,2300588853,L123057647,10/1/2022 12:45:00 AM,10,0.040,35012787756
4,2300588853,L123057647,10/1/2022 1:00:00 AM,10,0.045,35012787756
...,...,...,...,...,...,...
500275,2300588897,L108607371,9/30/2021 7:00:00 PM,10,1.242,35012790198
500276,2300588897,L108607371,9/30/2021 8:00:00 PM,10,1.202,35012790198
500277,2300588897,L108607371,9/30/2021 9:00:00 PM,10,1.186,35012790198
500278,2300588897,L108607371,9/30/2021 10:00:00 PM,10,1.150,35012790198


In [ ]:
# When using Jupyter Notebooks with a database file, you have to make sure to explicitly close it.
# Otherwise, you get a lock file that remains in an "open" state: look for `sql/electrc_brew.db.wal` to see if the database is currently locked
electric_brew.close()